<a href="https://colab.research.google.com/github/mjhoover1/Robust_MAML/blob/main/Torchmeta_MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, install the library torchmeta:

In [3]:
!pip install torchmeta

     |████████████████████████████████| 210 kB 26.9 MB/s 
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8219 sha256=9b59dd2ce40b67cb2d1ddc903c8952a4ae448e6ca254ebd971943a125ed41473
  Stored in directory: /root/.cache/pip/wheels/73/2b/f6/26e9f84153c25050fe7c09e88f8e32a6be3c7034a38c418319
Successfully built ordered-set


MAML example using torchmeta: Here is the model

In [4]:
import torch
import torch.nn as nn
from torchmeta.modules import (MetaModule, MetaSequential, MetaConv2d, MetaBatchNorm2d, MetaLinear)

def conv3x3(in_channels, out_channels):
  return MetaSequential(
      MetaConv2d(in_channels, out_channels, kernel_size=3, padding=1),
      MetaBatchNorm2d(out_channels, momentum=1., track_running_stats=False),
      nn.ReLU(),
      nn.MaxPool2d(2)
  )

class ConvolutionalNeuralNetwork(MetaModule):
  def __init__(self, in_channels, out_features, hidden_size=64):
    super(ConvolutionalNeuralNetwork, self).__init__()
    self.in_channels = in_channels
    self.out_features = out_features
    self.hidden_size = hidden_size

    self.features = MetaSequential(
        conv3x3(in_channels, hidden_size),
        conv3x3(hidden_size, hidden_size),
        conv3x3(hidden_size, hidden_size),
        conv3x3(hidden_size, hidden_size)
    )

    self.classifier = MetaLinear(hidden_size, out_features)

  def forward(self, inputs, params=None):
    features = self.features(inputs, params=self.get_subdict(params, 'features'))
    features = features.view((features.size(0), -1))
    logits = self.classifier(features, params=self.get_subdict(params, 'classifier'))
    return logits

Here are the utility functions for the MAML example:

In [6]:
from collections import OrderedDict

def get_accuracy(logits, targets):
    """Compute the accuracy (after adaptation) of MAML on the test/query points

    Parameters
    ----------
    logits : `torch.FloatTensor` instance
        Outputs/logits of the model on the query points. This tensor has shape
        `(num_examples, num_classes)`.
    targets : `torch.LongTensor` instance
        A tensor containing the targets of the query points. This tensor has 
        shape `(num_examples,)`.
        
    Returns
    -------
    accuracy : `torch.FloatTensor` instance
        Mean accuracy on the query points
    """
    _, predictions = torch.max(logits, dim=-1)
    return torch.mean(predictions.eq(targets).float())

Here is the training for the MAML example:

In [7]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()
device

device(type='cpu')

In [5]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
import logging

from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.utils.gradient_based import gradient_update_parameters

logger = logging.getLogger('example 1')

def train(folder, device, num_shots=5, num_ways=5, batch_size=16, num_batches=100, num_workers=1, hidden_size=64, step_size=.4, first_order=True, output_folder=None):
  dataset = omniglot(folder, shots=num_shots, ways=num_ways, shuffle=True, test_shots=15, 
                     meta_train=True, download=True)
  dataloader = BatchMetaDataLoader(dataset, batch_size=batch_size, shuffle=True, 
                                   num_workers=num_workers)
  model = ConvolutionalNeuralNetwork(1, num_ways, hidden_size=hidden_size)
  model.to(device=device)
  model.train()
  meta_optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  # Training loop
  with tqdm(dataloader, total=num_batches) as pbar:
    for batch_idx, batch in enumerate(pbar):

      model.zero_grad()

      train_inputs, train_targets = batch['train']
      train_inputs = train_inputs.to(device=device)
      train_targets = train_targets.to(device=device)

      test_inputs, test_targets = batch['test']
      test_inputs = test_inputs.to(device=device)
      test_targets = test_targets.to(device=device)
      
      outer_loss = torch.tensor(0., device=device)
      accuracy = torch.tensor(0., device=device)
      for task_idx, (train_input, train_target, test_input,
                     test_target) in enumerate(zip(train_inputs, train_targets,
                                                   test_inputs, test_targets)):
                       train_logit = model(train_input)
                       inner_loss = F.cross_entropy(train_logit, train_target)

                       model.zero_grad()
                       params = gradient_update_parameters(model,
                                                           inner_loss,
                                                           step_size=step_size,
                                                           first_order=first_order)
                       
                       test_logit = model(test_input, params=params)
                       outer_loss += F.cross_entropy(test_logit, test_target)

                       with torch.no_grad():
                         accuracy += get_accuracy(test_logit, test_target)
      outer_loss.div_(batch_size)
      accuracy.div_(batch_size)

      outer_loss.backward()
      meta_optimizer.step()

      pbar.set_postfix(accuracy='{0:.4f}'.format(accuracy.item()))
      if batch_idx >= num_batches: break

  # Save model
  if output_folder is not None:
    filename = os.path.join(output_folder, 'maml_omniglot_{0}shot_{1}way.th'.format(
        num_shots, num_ways))
    with open(filename, 'wb') as f:
      state_dict = model.state_dict()
      torch.save(state_dict, f)

In [ ]:
# train(folder, device, num_shots=5, num_ways=5, batch_size=16, num_batches=100, num_workers=1, hidden_size=64, step_size=.4, first_order=True, output_folder=None):
train("data", device)